## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather
0,0,Busselton,AU,-33.6500,115.3333,69.08,74,100,14.76,overcast clouds
1,1,Avarua,CK,-21.2078,-159.7750,80.65,83,20,6.91,few clouds
2,2,East London,ZA,-33.0153,27.9116,58.14,72,20,5.75,few clouds
3,3,Guerrero Negro,MX,27.9769,-114.0611,70.72,39,71,6.51,broken clouds
4,4,Dingle,PH,10.9995,122.6711,76.55,89,100,7.61,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input('What is the minimum temperature you would like for your trip?'))
max_temp = float(input('What is the maximum temperature you would like for your trip?'))

What is the minimum temperature you would like for your trip?70
What is the maximum temperature you would like for your trip?90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) & 
                                      (city_data_df['Max Temp'] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather
1,1,Avarua,CK,-21.2078,-159.7750,80.65,83,20,6.91,few clouds
3,3,Guerrero Negro,MX,27.9769,-114.0611,70.72,39,71,6.51,broken clouds
4,4,Dingle,PH,10.9995,122.6711,76.55,89,100,7.61,overcast clouds
5,5,Tulsipur,NP,28.1310,82.2973,89.71,18,0,7.14,clear sky
6,6,Rikitea,PF,-23.1203,-134.9692,79.84,70,6,15.32,clear sky
7,7,Bambous Virieux,MU,-20.3428,57.7575,80.22,94,40,8.05,scattered clouds
11,11,Haripur,PK,34.0000,72.9341,88.75,18,0,4.85,clear sky
12,12,Istmina,CO,5.1605,-76.6840,73.69,99,100,1.32,overcast clouds
13,13,Victoria,HK,22.2855,114.1577,79.95,36,0,8.99,clear sky
17,17,Souillac,MU,-20.5167,57.5167,77.38,94,40,8.05,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 278 entries, 1 to 685
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   City_ID          278 non-null    int64  
 1   City             278 non-null    object 
 2   Country          278 non-null    object 
 3   Lat              278 non-null    float64
 4   Lng              278 non-null    float64
 5   Max Temp         278 non-null    float64
 6   Humidity         278 non-null    int64  
 7   Cloudiness       278 non-null    int64  
 8   Wind Speed       278 non-null    float64
 9   Current Weather  278 non-null    object 
dtypes: float64(4), int64(3), object(3)
memory usage: 23.9+ KB


In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.clean_preferred_cities_df= preferred_cities_df.dropna()
clean_df = preferred_cities_df.dropna()
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 278 entries, 1 to 685
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   City_ID          278 non-null    int64  
 1   City             278 non-null    object 
 2   Country          278 non-null    object 
 3   Lat              278 non-null    float64
 4   Lng              278 non-null    float64
 5   Max Temp         278 non-null    float64
 6   Humidity         278 non-null    int64  
 7   Cloudiness       278 non-null    int64  
 8   Wind Speed       278 non-null    float64
 9   Current Weather  278 non-null    object 
dtypes: float64(4), int64(3), object(3)
memory usage: 23.9+ KB


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Weather", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Weather,Lat,Lng,Hotel Name
1,Avarua,CK,80.65,few clouds,-21.2078,-159.7750,
3,Guerrero Negro,MX,70.72,broken clouds,27.9769,-114.0611,
4,Dingle,PH,76.55,overcast clouds,10.9995,122.6711,
5,Tulsipur,NP,89.71,clear sky,28.1310,82.2973,
6,Rikitea,PF,79.84,clear sky,-23.1203,-134.9692,
7,Bambous Virieux,MU,80.22,scattered clouds,-20.3428,57.7575,
11,Haripur,PK,88.75,clear sky,34.0000,72.9341,
12,Istmina,CO,73.69,overcast clouds,5.1605,-76.6840,
13,Victoria,HK,79.95,clear sky,22.2855,114.1577,
17,Souillac,MU,77.38,scattered clouds,-20.5167,57.5167,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params['location'] = f'{lat},{lng}'
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params = params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
        print('Hotel not found... Skipping.')
        

Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.


In [10]:
hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 278 entries, 1 to 685
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   City             278 non-null    object 
 1   Country          278 non-null    object 
 2   Max Temp         278 non-null    float64
 3   Current Weather  278 non-null    object 
 4   Lat              278 non-null    float64
 5   Lng              278 non-null    float64
 6   Hotel Name       278 non-null    object 
dtypes: float64(3), object(4)
memory usage: 25.5+ KB


In [11]:
# 7. Drop the rows where there is no Hotel Name.
# Syntax to replace empty string,with NaN and then drop the NaN
# Https://stackoverflow.com/questions/29314033/drop-rows-containing-empty-cells-from-a-pandas-dataframe
hotel_df['Hotel Name'].replace('', np.nan, inplace = True)
hotel_df.dropna(subset=['Hotel Name'], inplace = True)
hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 253 entries, 1 to 685
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   City             253 non-null    object 
 1   Country          253 non-null    object 
 2   Max Temp         253 non-null    float64
 3   Current Weather  253 non-null    object 
 4   Lat              253 non-null    float64
 5   Lng              253 non-null    float64
 6   Hotel Name       253 non-null    object 
dtypes: float64(3), object(4)
memory usage: 15.8+ KB


In [12]:
hotel_df

,City,Country,Max Temp,Current Weather,Lat,Lng,Hotel Name
1,Avarua,CK,80.65,few clouds,-21.2078,-159.7750,Paradise Inn
3,Guerrero Negro,MX,70.72,broken clouds,27.9769,-114.0611,"Casa Laguna, Bed & Breakfast"
4,Dingle,PH,76.55,overcast clouds,10.9995,122.6711,SEGAYA PROPERTY
5,Tulsipur,NP,89.71,clear sky,28.1310,82.2973,Ekanta Hotel Pvt. Ltd
6,Rikitea,PF,79.84,clear sky,-23.1203,-134.9692,People ThankYou
...,...,...,...,...,...,...,...
674,Soyo,AO,77.22,overcast clouds,-6.1349,12.3689,Hotel Mpampa
677,Maningrida,AU,88.39,overcast clouds,-12.0667,134.2667,Dhukurrdji Lodge
678,Hubli,IN,82.44,clear sky,15.3500,75.1667,Hotel Swathi
679,Mogadishu,SO,85.87,broken clouds,2.0371,45.3438,Hotel Juba


In [13]:
# 8a. Create the output File (CSV)
output_data_file = '/Users/andreadarrah/Desktop/classwork/World_Weather_Analysis/Vacation_Search/WeatherPy_Vacation.csv'
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [16]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
locations = hotel_df[['Lat', 'Lng']]
max_temp = hotel_df['Max Temp']
fig = gmaps.figure(center = (30.0, 31.0), zoom_level = 1.5)
heat_layer = gmaps.heatmap_layer(locations, weights = max_temp,
                                dissipating = False, max_intensity = 300, point_radius = 4)
fig.add_layer(heat_layer)

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Weather}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [17]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))